In [1]:
import torch
from going_modular import going_modular
from pathlib import Path
import torch
import torchvision
import torchinfo
from torch import nn
from torchvision import transforms

/home/yuvrajsingh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # print(m.weight.data)
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [4]:
# class weight_initialization(nn.Module):
#     def __init__(
#         self,
#         weights_std_deviation = 0.2
#     ):
        
#         super().__init__()
        
#         self.wi = nn.init.normal_(mean=0.0, std=weights_std_deviation)
        
        
#     def forward(self, x):
#         initialized_weights = self.wi(x)
#         return initialized_weights

In [56]:
class Generator(nn.Module):
    def __init__(
        self,
        latent_vector_size = 100,
        no_of_channels = 3,
        kernel_size = (4,4),
        stride: int = 2,
        number_of_feature_maps: int = 64,
        padding: int = 1,
    ):
        
        super().__init__()

        # self.wi = weight_initialization()
        self.main = nn.Sequential(
            
            
            
            nn.ConvTranspose2d(latent_vector_size, number_of_feature_maps * 16 , kernel_size=kernel_size, stride=stride, padding=0),
            nn.BatchNorm2d(number_of_feature_maps * 16),
            nn.ReLU(),
            
            #shape = (...,1024, 4, 4)
            nn.ConvTranspose2d(number_of_feature_maps * 16, number_of_feature_maps * 8 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 8),
            nn.ReLU(),
            
            #shape = (..., 512, 8, 8)
            nn.ConvTranspose2d(number_of_feature_maps * 8, number_of_feature_maps * 4 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(number_of_feature_maps * 4),
            
             #shape = (..., 256, 16, 16)
            nn.ConvTranspose2d(number_of_feature_maps * 4, number_of_feature_maps * 2 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(number_of_feature_maps * 2),
            
             #shape = (..., 128, 32, 32)
            nn.ConvTranspose2d(number_of_feature_maps * 2, no_of_channels , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.Tanh()
            #shape = (..., 3, 64, 64)
        )
        
    def forward(self, x):
        x = self.main(x)
        return x

In [57]:
#Intializing the Generator instance
generator = Generator().to(device)

#Applying the weights transformation
generator.apply(weights_init)

#Printing the structure
print(generator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): ReLU()
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Tanh()
  )
)


In [58]:
from torchinfo import summary


summary(model=generator,
        input_size=(128, 100, 1, 1),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Generator (Generator)                    [128, 100, 1, 1]     [128, 3, 64, 64]     --                   True
├─Sequential (main)                      [128, 100, 1, 1]     [128, 3, 64, 64]     --                   True
│    └─ConvTranspose2d (0)               [128, 100, 1, 1]     [128, 1024, 4, 4]    1,639,424            True
│    └─BatchNorm2d (1)                   [128, 1024, 4, 4]    [128, 1024, 4, 4]    2,048                True
│    └─ReLU (2)                          [128, 1024, 4, 4]    [128, 1024, 4, 4]    --                   --
│    └─ConvTranspose2d (3)               [128, 1024, 4, 4]    [128, 512, 8, 8]     8,389,120            True
│    └─BatchNorm2d (4)                   [128, 512, 8, 8]     [128, 512, 8, 8]     1,024                True
│    └─ReLU (5)                          [128, 512, 8, 8]     [128, 512, 8, 8]     --                   --
│    └─ConvTranspo

In [93]:
class Discriminator(nn.Module):
    def __init__(
        self,
        no_of_channels = 3,
        kernel_size = (4,4),
        stride: int = 2,
        number_of_feature_maps: int = 64,
        padding: int = 1,
        lr_slope=0.2
    ):
        
        super().__init__()
        
         #shape = (..., 3, 64, 64)
        self.main = nn.Sequential(
            nn.Conv2d(no_of_channels, number_of_feature_maps * 2 , kernel_size=kernel_size, stride=stride, padding=padding),
            # nn.BatchNorm2d(number_of_feature_maps * 8),
            nn.LeakyReLU(negative_slope=lr_slope),
                
                #shape = (...,1024, 32, 32)
            nn.Conv2d(number_of_feature_maps * 2, number_of_feature_maps * 4 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 4),
            nn.LeakyReLU(negative_slope=lr_slope),
                
                #shape = (..., 512, 16, 16)
            nn.Conv2d(number_of_feature_maps * 4, number_of_feature_maps * 8 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 8),
            nn.LeakyReLU(negative_slope=lr_slope),
                
                #shape = (..., 256, 8, 8)
            # nn.Conv2d(number_of_feature_maps * 4, number_of_feature_maps * 8 , kernel_size=kernel_size, stride=stride, padding=0),
            # nn.BatchNorm2d(number_of_feature_maps * 8),
            # nn.LeakyReLU(negative_slope=lr_slope),
             #shape = (..., 128, 4, 4)
         )
        
    def forward(self, x):
        return self.main(x)

In [94]:
#Intializing the Discriminator instance
discriminator = Discriminator().to(device)


#Printing the structure
print(discriminator)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
  )
)


In [95]:
from torchinfo import summary


summary(model=discriminator,
        input_size=(128, 3, 64, 64),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Discriminator (Discriminator)            [128, 3, 64, 64]     [128, 512, 8, 8]     --                   True
├─Sequential (main)                      [128, 3, 64, 64]     [128, 512, 8, 8]     --                   True
│    └─Conv2d (0)                        [128, 3, 64, 64]     [128, 128, 32, 32]   6,272                True
│    └─LeakyReLU (1)                     [128, 128, 32, 32]   [128, 128, 32, 32]   --                   --
│    └─Conv2d (2)                        [128, 128, 32, 32]   [128, 256, 16, 16]   524,544              True
│    └─BatchNorm2d (3)                   [128, 256, 16, 16]   [128, 256, 16, 16]   512                  True
│    └─LeakyReLU (4)                     [128, 256, 16, 16]   [128, 256, 16, 16]   --                   --
│    └─Conv2d (5)                        [128, 256, 16, 16]   [128, 512, 8, 8]     2,097,664            True
│    └─BatchNorm2d

In [98]:
epochs = 5
beta_1 = 0.5
lr_optimizer = 0.0002
loss_fn = nn.BCELoss()  #BCELoss function
optimizer = torch.optim.Adam(params=, betas=(beta_1, 0.999), lr=lr_optimizer) 

batch_size = 128
latent_vector_size = 100

real_label = 1
fake_label = 0

#Noise
noise = torch.randn((batch_size, latent_vector_size, 1, 1))

TypeError: Adam.__init__() missing 1 required positional argument: 'params'

In [99]:
noise = torch.randn((batch_size, latent_vector_size, 1, 1))

NameError: name 'batch_size' is not defined

In [ ]:
#Training loop

